In [ ]:
# Computational_Python 2020
#Name: Roberto Heredia Fonseca

In [ ]:
# Reading and plotting demand 2018

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


#change cm to inch for figure size

def cm2inch(*tupl):
    inch = 2.54
    if isinstance(tupl[0], tuple):
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)

#Change font siz
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=10)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize= 6)            # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
#reading de excel file
#pass a dictionary with columns of day and time since there are three days missing in the data. Or it is possible to create a date_range with a space of freq of 30T (30min)

df=pd.read_excel('Informacion Demanda horaria 2018.xlsx', parse_dates={'date':['Fecha', 'Hora']})
#df.head(5)

df.rename(columns ={'Hidraulica [MW]': 'Hydropower', 'Renovable [MW]': 'Non-hydro res', 'Termica [MW]': 'Thermal'}, inplace=True) 
#df.info()

In [ ]:
df['day'] = df['date'].dt.weekday_name

df['month'] = df['date'].dt.month_name()

df['time'] = df['date'].dt.time
df.head(5)

In [ ]:
plt.plot('date', 'Total [MW]', data=df, linestyle='solid', linewidth=.25)
plt.ylabel('Demand [MW]')
plt.savefig('Demand_total.png', dpi=300, bbox_inches='tight', figsize=cm2inch((7.5, 10)))
plt.show()

#Plot demand by three main tech

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, figsize=cm2inch((7.5, 10)))
#fig.suptitle("Energy demand 2018")

ax1.plot('date','Hydropower', data=df, linestyle='none',marker='.',markersize=1, color='#16365C')
ax1.grid(b=bool)
ax1.minorticks_on()

ax2.plot('date','Non-hydro res', data=df, linestyle='none',marker='.',markersize=1, color="green")
ax2.grid(b=bool)
ax2.minorticks_on()

ax3.plot('date','Thermal', data=df, linestyle='none',marker='.',markersize=1, color='#B7423F')
ax3.grid(b=bool)
ax3.minorticks_on()

ax2.set_ylabel("Demand [MW]")

# Define the date format
myFmt = mdates.DateFormatter('%b')
ax1.xaxis.set_major_formatter(myFmt)
ax2.xaxis.set_major_formatter(myFmt)
ax3.xaxis.set_major_formatter(myFmt)

#adjust distance between subplots
plt.subplots_adjust(top=1, hspace=0.4)

#legend
plt.figlegend(loc='upper right', bbox_to_anchor=(0.6, 0.62))
plt.rcParams.update({'legend.labelspacing':0.15})

plt.savefig('Ec demand 2018.png',dpi=300, bbox_inches='tight')
#plt.show()

#Plot demand by threee timestamps

In [ ]:
# possible to use resample or groupby
# resample() groups rows by some time or date information,
# .groupby() groups rows based on the values in one or more columns.

monthly = pd.DataFrame(df.resample('M', on='date')['Total [MW]'].mean())
x = monthly.index
y = monthly['Total [MW]'] 

hour = pd.DataFrame(df.groupby(df['date'].dt.hour)['Total [MW]'].mean())
x1 = hour.index
y1 = hour['Total [MW]'] 

day = pd.DataFrame(df.groupby(df['date'].dt.weekday)['Total [MW]'].mean())
#he day of the week with Monday=0, Sunday=6.
x2 = day.index
y2 = day['Total [MW]'] 

In [ ]:
fig_ts,(ax1_m, ax2_d, ax3_h) = plt.subplots(nrows=3, figsize= cm2inch((7.5,10)))
#fig_ts.suptitle("Average energy demand 2018")

ax1_m.plot(x,y, linestyle='solid',marker='.',markersize=5, color='#16365C')
ax1_m.grid(b=bool)
#ax1_m.minorticks_on()

ax1_m.xaxis_date()
myFmt = mdates.DateFormatter('%b')
ax1_m.xaxis.set_major_formatter(myFmt)

ax2_d.plot(x2,y2, linestyle='solid',marker='.',markersize=5, color="green")
ax2_d.grid(b=bool)
ax2_d.minorticks_on()

ax3_h.plot(x1,y1, linestyle='solid',marker='.',markersize=5, color='#B7423F')
ax3_h.grid(b=bool)
ax3_h.minorticks_on()

ax2_d.set_ylabel("Average demand [MW)")


#adjust distance between subplots
plt.subplots_adjust(top=1, hspace=0.4)

#legend
fig_ts.legend([ax1_m, ax2_d, ax3_h],
           labels= ["Month", "DoW", "time"],      # The line objects
           borderaxespad=0.1,                    # Small spacing around legend box
           loc=(0.26, 0.46)
           )
#plt.savefig('ts.pdf',dpi=300)
plt.savefig('ts.png',dpi=400,  bbox_inches='tight')
plt.show()

#Joinning all data frames by columns

In [ ]:
df_h = df[['date','Hydropower', 'day', 'time', 'month']].rename(columns={'Hydropower':'Demand'})
df_h['Tech']= 'Hydropower'
#df_h

df_t = df[['date','Thermal', 'day', 'time','month']].rename(columns={'Thermal':'Demand'})
df_t['Tech']= 'Thermal'
#df_t

df_n = df[['date','Non-hydro res', 'day', 'time','month']].rename(columns={'Non-hydro res':'Demand'})
df_n['Tech']= 'Non-hydro res'
#df_n

In [ ]:
df1 = pd.concat([df_h, df_t,df_n], axis=0, ignore_index='True')
df1.head(5)

In [ ]:
#change x to day, month or time
f2 =px.box(df1, x="day", y='Demand', facet_row='Tech', color='Tech')
f2.update_yaxes(matches=None)
f2.update_layout()
f2.show()
#f2.write_html("rh_demand_day.html")

# Reading data (more than 300K data points)
# Capacity factors (cf) calculation


In [ ]:
# Read generation by year
# change the year #
df = pd.read_excel('EC generation 2018.xlsx',sheet_name='rpConsolidadoEnergiaProducida',skiprows=0,skipfooter=0,usecols='A,G,M,N')

# Read the list of power plants and capacity [MW]
df_pp = pd.read_excel('Power plants.xlsx') 
df_pp = df_pp.to_numpy()

#data frame for capacity factors
df_cf = pd.DataFrame(np.zeros((0,0)))
data = np.zeros((12,1))

# array 'days' contains every month and #days
days = np.array([['Ene',31],['Feb',28], ['Mar',31],['Abr',30],['May',31],['Jun',30],['Jul',31],['Ago',31],['Sep',30],['Oct',31],['Nov',30],['Dic',31]])

In [ ]:
# defining a function that first filter by power plant, system
# then, it will calculate the capacity factor by each month of the year
def c_f(power_plant,capacity):
    df1 = df.loc[(df['Central'] == power_plant) & (df['Sistema'] == 'S.N.I')].drop(columns=['Sistema'])
    for i in range(1, len(days)):
        data[i-1,0] = df1.loc[df['Mes'] == days[i-1,0]].sum(axis = 0, skipna = True)[2]  # finding sum over 
   
    #converting the array to a data frame
    a = pd.DataFrame(data)
    a = a.rename(columns={0:"Me"}) # Me montly energy

    # Declare a list that is to be converted into a column 
    d = [31,28,31,30,31,30,31,31,30,31,30,31] 
    a['d'] = d 

    ###calculating the cf
    a[power_plant] = (a['Me']/(a['d']*capacity*24)).round(3)

    ###Asigning the cf to cf matrix
    df_cf[power_plant] = a[power_plant]

In [ ]:
# for loop to read every hydropower plant and its capacity from df_pp 
for i in range(1,len(df_pp)):
    pp = df_pp[i-1,0]
    cp = df_pp[i-1,1]
    c_f(pp,cp)

df_cf.head(5)

In [ ]:
#print csv
df_cf.to_csv('c_factors_2018.csv')

# Results visualization

In [ ]:
# read results of the optimization
PbT = pd.read_csv('ProductionByTechnology.csv')
PbT.head(5)
#PbT.info()

In [ ]:
# read the data frame to filter PbT
tech = pd.read_excel('Tech.xlsx')
tech.head(5)

#define the color by tech
t_colors = dict(zip(tech['t_color'].dropna(), tech['hex'].dropna()))

In [ ]:
# filtered data frame
PbT_f = PbT[PbT['f'].isin(tech['fuels'].dropna()) & PbT['t'].isin(tech['t']) & PbT['ProductionByTechnology']>0]
PbT_f = PbT_f.reset_index(drop=True)
PbT_f.head(5)

In [ ]:
# function to changes the code names of the generation technologies to easy names
def tech_names(x):
    t = x['t'].to_numpy()
    tech_t = tech['t'].to_numpy()
    tech_t_n = tech['t_names'].to_numpy()

    l = len(x)
    l_n = len(tech_t)

    for j in range(0,l):
        for i in range(0,l_n):
            if t[j] == tech_t[i]:
                t[j] = tech_t_n[i]
    
    x = x.append(pd.DataFrame(t, columns=['t'])) 

In [ ]:
# calling the function to change names in a data frame.
tech_names(PbT_f)
PbT_f.head(5)

In [ ]:
#groupby technology and year
#This is a important step to use plotly otherwise it will plot every single bar

PbT_f = PbT_f.groupby(['t', 'y'])['ProductionByTechnology'].sum().reset_index()
PbT_f.head(5)

In [ ]:
fig_PbT = px.bar(PbT_f, x='y', y='ProductionByTechnology', color='t', barmode='relative',
opacity=1, color_discrete_map=t_colors)

fig_PbT.update_layout(title_text="Generation by Technology",
 xaxis=dict(title_text='year'),
 yaxis=dict(title_text='Generation [PJ]'),
 plot_bgcolor='rgb(255,255,255)',
 font_size=10) 

#fig_PbT.write_image('Generation_be_tech.jpg', width= 600, height= 600)

fig_PbT.show()
#fig_PbT.write_html("ProductionByTech.html")